In [4]:
from TTS.api import TTS

# Initialize the TTS model
tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2")

# Text to be converted to speech
text = "Hello, this is a test of text-to-speech using a custom voice."

# Path to your custom voice audio file (should be 6-10 seconds long)
custom_voice_path = "path/to/your/custom_voice_sample.wav"

# Generate speech with the custom voice
tts.tts_to_file(
    text=text,
    file_path="output_speech.wav",
    speaker_wav=custom_voice_path,
    language="en"
)

 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
